# Data Analysis - (Social network and property calculator)
### Enrico Gavagnin

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import py_fort_myrmidon as fm
import datetime
import networkx as nx
import networkx.algorithms.community as nxc
import statistics
import scipy.stats as stats
import os
from datetime import datetime, date, timedelta
import pickle
import warnings
import json

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Set working directory and open list of myrmidon files
working_dir = '/media/ll16598/EG_DATA-7/NTM/'
myrm_list = sorted([ s for s in os.listdir(working_dir) if s[15:17] == '_z'])
myrm_list = [m for m in myrm_list if '_nn' in m]


## Test mode (optional)

To reduce the computational cost of the script, turn on the test mode option. This will run the analysis only on the first 2 myrmidon files. 

In [3]:
test_mode = False
#test_mode = True

## Home-range communities heatmap plotting

## Main script

This script loops through all the myrmidon files. For each replicate it construct the social network, compute all the network properties and save them in a dataframe called prop_df.

In [4]:
save_G=True
save_df=False
compute_props=False

In [5]:
from collections import Counter
def compute_space_use(exp, start, end, min_cum_duration, frm_rate, link_type, nest_focus):
    TimeToFrame = {fm.Time.ToTimestamp(frm[0].FrameTime): i + 1 
               for i, frm in enumerate(fm.Query.CollideFrames(exp,start=start,end=end))}
    zone_durations=[]
    zone_ids=[]
    zone_ant_ids=[]
    zone_starts=[]
    zone_ends=[]
    #multilayer component. First necessary to compute trajectories
    trajectories=fm.Query.ComputeAntTrajectories(exp, start=start,end=end, computeZones=True)
    for t in range(0,len(trajectories)):
        tr=trajectories[t]
        ant_id=tr.Ant
        pos=tr.Positions
        t_start=tr.Start
        duration=(tr.End()-tr.Start).Seconds()
        #print(ant_id)
        for i in range(0,len(pos)):
            p=pos[i]
            zone=p[4]
            if zone==2:
                continue
            start=p[0]
            e = pos[i+1][0] if i+1 < len(pos) else duration
            length=e-start
            zone_durations.append(length)
            zone_ids.append(zone)
            zone_ant_ids.append(ant_id)
            try:
                zone_starts.append(TimeToFrame[fm.Time.ToTimestamp(t_start+start)] )
            except KeyError:
                zone_starts.append(TimeToFrame[fm.Time.ToTimestamp(t_start)] )
            zone_ends.append(TimeToFrame[fm.Time.ToTimestamp(t_start+length)] )
    df = pd.DataFrame({
    'zone_durations': zone_durations,
    'zone_ids': zone_ids,
    'zone_ant_ids': zone_ant_ids,
    'zone_starts': zone_starts,
    'zone_ends': zone_ends
    })
    return df


In [10]:
complete_list = sorted([ s for s in os.listdir(working_dir+'space_use/')])

In [12]:
str('_'+str(time_win_h)+'_'+str(tw + 1)+'_'+myrm_file+ "_.csv")

'_3_22_EG_NTM_s34_DEHb_zones_luke_2023_nn.myrmidon_.csv'

In [13]:
## ========= PARAMETERS========= 
# Frame rate
frm_rate = 6

# max gap for different interactions (s)
max_gap = 10

# minimum interaction weight (OPTIONAL, set=0 if not wanted)
min_cum_duration = 0 

# Read file with modes of communities number computed   
if compute_props==True:
    mode_communities_dic = pd.read_pickle(r'data/mode_communities.pkl')

# ========= Edges weights type ==========
# Decide what type of weight for the edges of the social network 
#     'length_inter': cumulative interaction time (s))    
#     '#inter': number of interactions    

link_type = '#inter'

# ========= Nest focus zone ==========
# Set weather to only consider interactions happening within the nest zone
#     'True': Nest only
#     'False': All interactions considered
nest_focus = True
#how long to compute space use over
hours_g=24

data_saving_name = 'data/polish_data_test' + link_type + '_NF_F.pkl'


if test_mode:
    myrm_list = myrm_list[20:24]

    
working_dir+'space_use/'+'_'+str(time_win_h)+'_'+str(tw + 1)+'_'+myrm_file+ "_.csv"
# Loop through the myrmidon files
for myrm_file in myrm_list:

    # Skip replicate 41 (major escape), and all the replicates of the first 3 blocs (different tag orientation)
    if int(myrm_file[8:10])==41 or int(myrm_file[8:10])<13: 
        continue

    print(myrm_file)

    # Open experiment file
    exp = fm.Experiment.Open(working_dir + myrm_file)

    
    # Read the start date of the time window to be considered for the analysis
    # (ie the day before the experiment has been terminated)
    start_date = (fm.Time.ToDateTime(fm.Query.GetDataInformations(exp).End) +
                 timedelta(days = -1)).strftime("%Y-%m-%d")
    # Compute start/end of the time window
    start = fm.Time(datetime.fromisoformat(start_date + 'T09:00:00'))  
    end = start.Add(fm.Duration(60 * 60 * hours_g * 10**9))
    for time_win_h in [3]:

        print('Time_win:' + str (time_win_h) + 'h')
        # Loop through non-overlapping time windows of the given length
        for tw in range(0,25 - time_win_h, time_win_h):#
            if str('_'+str(time_win_h)+'_'+str(tw + 1)+'_'+myrm_file+ "_.csv") in complete_list:
                continue
            print('\rtime_slot: #' + str(tw + 1), end="")
            
            # Compute start/end of the time window
            s = start.Add(fm.Duration(tw * 60**2 * 10**9))
            e = start.Add(fm.Duration((tw + time_win_h) * 60**2 * 10**9))
    # Compute network
            space_use = compute_space_use(exp, s, e, min_cum_duration, frm_rate, link_type, nest_focus=nest_focus)
            space_use.to_csv(working_dir+'space_use/'+'_'+str(time_win_h)+'_'+str(tw + 1)+'_'+myrm_file+ "_.csv")



EG_NTM_s13_DEHa_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s13_DEHb_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s14_MODa_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s14_MODb_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s15_DENa_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s15_DENb_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s16_DIAa_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s16_DIAb_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s17_MODa_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s17_MODb_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s18_DENa_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s18_DENb_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s19_DIAa_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s19_DIAb_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s20_DEHa_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s20_DEHb_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s21_DIAa_zones_luke_2023_nn.myrmidon
Time_win:3h
EG_NTM_s21_DIAb_zones_luke_2023_nn.myrmidon
Time

Computing ant trajectories:  99%|███▉| 179/180 [00:05<00:00, 33.75tracked min/s]


time_slot: #4

Computing ant trajectories:  99%|███▉| 179/180 [00:05<00:00, 33.32tracked min/s]


time_slot: #7

...

time_slot: #16

Computing ant trajectories:  99%|███▉| 179/180 [00:05<00:00, 31.40tracked min/s]


time_slot: #19

Computing ant trajectories:  99%|███▉| 179/180 [00:05<00:00, 31.79tracked min/s]


KeyboardInterrupt: 

I now have multilayer networks where the zone attribute is the duration ant has spent in zone. Next to get the rate of switching and max density I need to get a df with the number of ants in a given zone at a time. I could get heatmap entropy also. And I could get number of interactions at a given time - although can be done with the temporal

In [8]:
            space_use.to_csv(working_dir+'space_use/'+'_'+str(time_win_h)+'_'+str(tw + 1)+myrm_file+ "_.csv")


## Computing mode of communitites
This script compute the mode of the number of communities abtained with unsupervised greedy algorithm and save the result in a file called mode_communities.pkl

In [ ]:
# Mode of communities number with greedy algorithm 

# ========== FILTERING ============
reps_discarded = [20,41]
mode_communities_dic = {}

for data_file_name in ['data/prop_data_12_6_4_3_2_1_#inter_nest_21042022.pkl']:

    df = pd.read_pickle(data_file_name)
    
    if '_#inter_' in data_file_name:
        link_type = '#inter'
    elif '_length_inter_'in data_file_name:
        link_type = 'length_inter'
    
    mode_communities_dic[link_type] = {}

    for exp in ['MOD', 'DIA', 'DEN', 'DEH']:
        mode_communities_dic[link_type][exp] = {}
        
        for time_win in [1, 2, 3, 4, 6, 12]:
            
            df_filt = df.loc[(df.time_win==3600 * time_win) &  
                            (df.exp.isin([exp+'a',exp+'b'])) & 
                            (~df.rep.isin(reps_discarded))]
            
            mode_communities_dic[link_type][exp][time_win * 3600] = stats.mode([len(com) for com in df_filt.cMOD_communities]).mode[0]
            
a_file = open('data/mode_communities.pkl', "wb")
pickle.dump(mode_communities_dic, a_file)

## Connectivity Analysis

This script contains an auxiliary function which compute the size of the largest connected component as a funciton of the size time window of aggregation (Percolation analysis) 

NOT WELL DOCUMENTED

In [ ]:
# Time vs Connectivity

def compute_Gcc(exp, start, end):
    
    # Number of ants
    N_ants = len(exp.Ants)    

    # initialise adj-matrix
    adj_mat = np.zeros((N_ants, N_ants))
    
    Gcc = [[sorted(nx.connected_components(nx.Graph(adj_mat)), key=len, reverse=True)],
           [0]]

    # Populate network
    for i in fm.Query.ComputeAntInteractions(exp,start=start,end=end,maximumGap=fm.Duration(max_gap*10**9),
                                            reportFullTrajectories= False)[1]:
        # Focus on Nest zone (id=1)
        if (1 in i.Trajectories[1].Zones) & (adj_mat[i.IDs[0]-1, i.IDs[1]-1] == 0):
            
            adj_mat[i.IDs[0]-1, i.IDs[1]-1] = 1           
         
            if i.Start.After(start.Add(fm.Duration((Gcc[1][-1] + 1) * 10**9))):
    
                Gcc[0].append(sorted(nx.connected_components(nx.Graph(adj_mat)), key=len, reverse=True))
                Gcc[1].append(Gcc[1][-1] + 1)
                    
    # network build
    return   Gcc

## ============= LOOP MYRMIDON =============== 

Gcc_df = pd.DataFrame(columns=['rep', 'exp', 'Gcc'])
max_gap = 10
for myrm_file in myrm_list:

    if int(myrm_file[8:10])==41 or int(myrm_file[8:10])<13: 
        continue

    print(myrm_file)

    # Open experiment file
    exp = fm.Experiment.Open(working_dir + myrm_file)

    ## ------ Time window ------ 
    start_date = (fm.Time.ToDateTime(fm.Query.GetDataInformations(exp).End) +
                 timedelta(days = -1)).strftime("%Y-%m-%d")

    start_time = fm.Time(datetime.fromisoformat(start_date + 'T09:00:00'))  
    Gcc = compute_Gcc(exp,start_time,start_time.Add(fm.Duration(12 * 60 * 60 * 10**9)))
    Gcc_df = Gcc_df.append({'rep': int(myrm_file[8:10]), 'exp': myrm_file[11:15], 'Gcc': Gcc}, ignore_index=True)
    
# Save
a_file = open("data/Gcc_NEST.pkl", "wb")
pickle.dump(Gcc_df, a_file)
a_file.close()


In [ ]:

plot_fld_path = 'plots/Connectivity_analysis/'


for exp in ['DIA', 'MOD', 'DEH', 'DEN']:
    
    # Reading dataframe of connectivity
    Gcc_df = pd.read_pickle('data/Gcc_NEST.pkl')

    
    #Filter for the experiement of interest
    Gcc_df = Gcc_df[Gcc_df.rep != 20]
    Gcc_df = Gcc_df[[e[0:3] == exp for e in Gcc_df.exp]]
    Gcc_df.sort_values(['rep','exp'])
    connect_df = pd.DataFrame(columns=['rep', 'nest', 'time (min)', 'size_GCC', '#CC'])

    # treshold minimal partition size
    tresh_part = 2

    # Reformatting dataframe
    for row in range(len(Gcc_df)):
        df_aux = pd.DataFrame({'rep': [Gcc_df.rep.iloc[row]] * len(Gcc_df.Gcc.iloc[row][0]),
                              'nest': [Gcc_df.exp.iloc[row]] * len(Gcc_df.Gcc.iloc[row][0]),
                              'time (min)': Gcc_df.Gcc.iloc[row][1],
                              'size_GCC': [len(cc[0]) for cc in Gcc_df.Gcc.iloc[row][0]],
                              '#CC': [sum([len(p) > tresh_part for p in cc]) for cc in Gcc_df.Gcc.iloc[row][0]]})
        connect_df = pd.concat([connect_df, df_aux], ignore_index=True)

    # Plotting    
    sns.set(font_scale = 1.1)
    fig, axs = plt.subplots(1, 1, figsize=(15,10))
    title = 'Connectivity_exp_' + exp + ', reps: ' + str(int(len(Gcc_df)/2)) 
    supt = plt.suptitle(title, fontweight="bold")
    # Plot 1 (size GCC)
    sns.lineplot(data=connect_df, x="time (min)", y="size_GCC", hue="nest", ax=axs)
    axs.set_title('Size connected component')
    plt.tight_layout()
    axs_i_r = axs.inset_axes([0.6,0.1,0.35,0.4])

    # Plot 1 (number CC)
    sns.lineplot(data=connect_df, x="time (min)", y="#CC", hue="nest", ax=axs_i_r)
    axs_i_r.set_title('Number of connected components of size > ' + str(tresh_part))
    axs_i_r.set_xlim([1,40])
    axs_i_r.set_ylim(bottom=1)
    plt.tight_layout()

    # ====================================
    # Difference in size of GCC
    diff_connect_df = pd.DataFrame(columns=['rep', 'time (min)', 'diff_GCC'])

    def diff_list_GCC(A, B):
        A_len = [len(a) for a in A]
        B_len = [len(b) for b in B]
        return [A_len[i] - B_len[i] for i in range(min(len(A), len(B)))]

    # Reformatting dataframe
    for row in range(0, len(Gcc_df) -1, 2):
        diff_GCC = diff_list_GCC(Gcc_df.Gcc.iloc[row][0], Gcc_df.Gcc.iloc[row + 1][0])
        df_aux = pd.DataFrame({'rep': [Gcc_df.rep.iloc[row]] * len(diff_GCC),
                              'time (min)': range(len(diff_GCC)),
                              'diff_GCC': diff_GCC})
        diff_connect_df = pd.concat([diff_connect_df, df_aux], ignore_index=True)

    # Plotting    
    # Plot 3 (diff size GCC)
    axs_i_l = axs.inset_axes([0.2,0.1,0.35,0.4])
    sns.lineplot(data=diff_connect_df, x="time (min)", y="diff_GCC", ax=axs_i_l)
    axs_i_l.plot([0, 720], [0, 0], ':r')
    axs_i_l.set_title('Difference in size of connected component (' + Gcc_df.exp.iloc[0] + ' - ' + Gcc_df.exp.iloc[1] + ')')
    plt.tight_layout()

    file_name = 'Connectivity_exp_' + exp 

    #Savefig
    plt.savefig(plot_fld_path + file_name + '.png', facecolor='white', transparent=False)

